# Tfidfの実装
- 古典的な手法で分類

# ライブラリの読み込み

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from scikitplot.metrics import plot_confusion_matrix, plot_roc

import MeCab

import re

# データの読み込み

In [2]:
DATA_DIR = "../data/"

train_raw = pd.read_csv(f"{DATA_DIR}raw/train.csv")
test_raw = pd.read_csv(f"{DATA_DIR}raw/test.csv")

train = train_raw.copy()
test = test_raw.copy()

# 改行文字等削除

In [3]:
def clean_text(text):
    new_text = []

    for line in text:
        # 半角全角英数字除去
        line = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", line)
        # 記号もろもろ除去
        line = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", line)
        
        line = line.strip("\n")
        line = line.replace("\n", "")
        line = line.rstrip("\u3000")
        line = line.replace("\u3000", "")

        new_text.append(line)
    new_text = "".join(new_text)
    return new_text

In [4]:
train["text"] = train["text"].apply(clean_text)

In [5]:
def add_char_num(text):
    char_num = []

    for line in text:
        char_num.append(len(line))
    return char_num

train["char_num"] = add_char_num(train["text"])

# 分かち書き（MeCab）

In [6]:
m = MeCab.Tagger("-Owakati")

text_owakati = []
for line in train["text"]:
    splited_line = m.parse(line)    
    
    # 改行文字削除
    splited_line = splited_line.replace("\n", "")
    
    text_owakati.append(splited_line)

train["text_owakati"] = text_owakati
train["word_num"] = train["text_owakati"].apply(lambda x: len(x.split()))

train.head()

,id,source,text,label,char_num,text_owakati,word_num
0,80074aa43,news4vip,まともに相手されてない人との関係なんてそんな大事にするものか,0,30,まとも に 相手 さ れ て ない 人 と の 関係 なんて そんな 大事 に する もの か,18
1,6378fea6b,livejupiter,最近はアヘアヘ マンやない ｲｲ∀ ｲｸﾅｲ,0,25,最近 は アヘアヘ マン や ない ｲｲ ∀ ｲｸﾅｲ,9
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな,1,45,日本 人 と し て 生まれ て も 無能 な 低 学歴 って 分かっ たら 日本 人 の ...,28
3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよなガチャから引いたら圧倒的歓喜レベルやで,0,46,よくよく 思え ば 川上 は 配布 に し たら とんでも なく 有能 だ よ な ガチャ ...,25
4,51e4036bf,newsplus,押井は原作レイプの専門家だから原作マンガの真意を誤解させることに関してはプロだがそれ以外には...,0,54,押井 は 原作 レイプ の 専門 家 だ から 原作 マンガ の 真意 を 誤解 さ せる ...,34


# ベクトル化

In [7]:
X = train["text"]
y = train['label'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)

In [9]:
def tokenize(text):
    """ MeCab で分かち書きした結果をトークンとして返す """
    wakati = MeCab.Tagger("-O wakati")
    return wakati.parse(text)

In [10]:
vectorizer = TfidfVectorizer(tokenizer=tokenize)
X_train = vectorizer.fit_transform(X_train)

# ナイーブベイズ

In [11]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

X_test = vectorizer.transform(X_test)

In [12]:
print(clf.score(X_train, y_train)) 
print(clf.score(X_test, y_test)) 

0.9405328258801142
0.9467680608365019
